In [ ]:
!pip install InstructorEmbedding

In [ ]:
!pip install -U sentence-transformers
# imports
import torch
from InstructorEmbedding import INSTRUCTOR

In [ ]:
!pip install langchain

In [ ]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (StreamingStdOutCallbackHandler)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
!pip install PyPDF2
!pip install faiss-cpu

In [ ]:
# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

### Prepare the Data

In [ ]:
!wget -O dataset.zip https://zenodo.org/record/7152317/files/dataset.zip?download=1


In [ ]:
!unzip dataset.zip

In [ ]:
judgement_path1 = '/content/dataset/IN-Abs/train-data/judgement/1.txt'
with open(judgement_path1, 'r') as file:
    judgement_1 = file.read()


In [ ]:
judgement_1

In [ ]:
# ###Prepare the text information
# #### Data from here : https://ncert.nic.in/textbook/pdf/leac203.pdf
# pdf_docs = ['leac203.pdf']
# text = ""
# for pdf in pdf_docs:
#     pdf_reader = PdfReader(pdf)
#     for page in pdf_reader.pages:
#         text += page.extract_text()
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(judgement_1)
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
#model_name = 'hkunlp/instructor-xl'
model_kwargs = {"device": "cpu"}

#embeddings = HuggingFaceInstructEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
embeddings = HuggingFaceEmbeddings(model_name = model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_texts(chunks, embeddings)
# save embeddings in local directory
vectorstore.save_local("legal_1")

# load from local
db = FAISS.load_local("legal_1/", embeddings=hf_embedding)

In [ ]:
query = "Which Acts were involved"
search = db.similarity_search(query, k=2)

# Using list comprehension
page_contents = [doc.page_content for doc in search]



In [ ]:
page_contents[0]

'With great respect, it may be pointed out that all charges in a way may be or are of a variable and contingent na ture.\nIf no default is made, no charge is ever enforceable and whenever there is a charge, it can be increased or reduced during the year either by payment or by additional borrowing.\nIn Moss Empires Ltd. vs Inland Revenue Commissioners (1) it was held by the House of Lords that the fact that certain payments were contingent and variable in amount did not affect their character of being annual payments and that the word, "annual" must be taken to have the quality of being recurrent or being capable of recurrence.\nIn Cunard \'s Trustees vs Inland Revenue Commissioners (2) it was held that the payments were capable of being recur rent and were therefore annual payments within the meaning of schedule D, case III, rule 1 (1), even though they were not necessarily recurrent year by year and the fact that they varied in amount was immaterial.'

In [ ]:
page_contents[1]

'Procedure has also been prescribed for recovery of the instalments by presentment of a bill, a notice of demand and then distress, and sale.\nFinally section 212 provides as follows : "Property taxes due under this Act in respect of any building or land shall, subject to the prior payment of the land revenue, if any, due to the provincial ,Government thereupon, be a first charge . . upon the said build ing or land . " It creates a statutory charge on the building.\nUrban immove able property tax is leviable under section 22 of Part VI of the Bombay Finance Act, 1932,on the annual letting value of the property.\nThe duty to collect the tax is laid on the municipality and it does so in the same manner as in the case of the municipal property tax.\nSection 24 (2) (b) is in terms similar to section 212 of the Bombay Municipal Act.\nIt makes the land or the building security for the payment of this tax also.'